In [ ]:
import httpx
from selectolax.parser import HTMLParser
from dataclasses import dataclass, asdict
import csv


@dataclass
class placename_information:
    # title: str | None
    placename: str = "No table on page"
    various_spellings: str = "No table on page"
    authority: str = "No table on page"
    situation: str = "No table on page"
    description: str = "No table on page"


def get_html(page):
    url = f"https://scotlandsplaces.gov.uk/digital-volumes/ordnance-survey-name-books/lanarkshire-os-name-books-1858-1861/lanarkshire-volume-01/{page}"
    response = httpx.get(url)
    return HTMLParser(response.text)


def parse_placename(html):
    try:
        name = html.css("div.well")
        results = []
        for item in name:
            new_item = placename_information(
                placename=item.css_first("td").text().strip(),
                various_spellings=item.css_first("td:nth-child(2)").text().replace('\n', '').strip(),
                authority=item.css_first("td:nth-child(3)").text().strip(),
                situation=item.css_first("td:nth-child(4)").text().strip(),
                description=item.css_first("td:nth-child(5)").text().strip(),
            )
            results.append(asdict(new_item))
        return results
    except AttributeError as error:
        return results


def to_csv(res):
    with open("name_books.csv", "a") as file:
        writer = csv.DictWriter(
            file,
            fieldnames=[
                "placename",
                "various_spellings",
                "authority",
                "situation",
                "description",
            ],
        )
        writer.writerows(res)


def main():
    for x in range(1, 10):
        html = get_html(x)
        print(html.css_first("title").text())
        res = parse_placename(html)
        print(res)
        to_csv(res)


if __name__ == "__main__":
    main()
